In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [ ]:
if not os.path.exists("micro_diffusion"):
    !git clone https://github.com/SonyResearch/micro_diffusion.git

In [ ]:
%pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu124

In [ ]:
%pip install "mosaicml[tensorboard, wandb]"

In [ ]:
%pip install \
    accelerate \
    diffusers \
    huggingface_hub \
    transformers \
    timm \
    open_clip_torch \
    easydict \
    einops \
    mosaicml-streaming \
    torchmetrics \
    tqdm \
    pandas \
    fastparquet \
    omegaconf \
    datasets \
    hydra-core \
    beautifulsoup4

In [ ]:
%pip install -e micro_diffusion --no-deps

In [ ]:
%cd /workspaces/micro_diffusion/micro_diffusion

import os
import micro_diffusion

# パスを指定
USER_ROOT = os.path.expanduser("~")
CACHE_DIR = os.path.join(USER_ROOT, ".cache", "micro_diffusion")
DATA_DIR = os.path.join(CACHE_DIR, "data")
MODEL_DIR = os.path.join(CACHE_DIR, "models")

COMPRESSED_DIR = os.path.join(DATA_DIR, 'compressed')
RAW_DIR = os.path.join(DATA_DIR, 'raw')

TRAIN_IMGS_DIR = os.path.join(RAW_DIR, 'train', 'imgs')
VALID_IMGS_DIR = os.path.join(RAW_DIR, 'valid', 'imgs')
TEST_DIR = os.path.join(RAW_DIR, 'test')

# ディレクトリ作成
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(COMPRESSED_DIR, exist_ok=True)
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(TRAIN_IMGS_DIR, exist_ok=True)
os.makedirs(VALID_IMGS_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)

In [ ]:
# !python download.py --datadir ./datadir/jdb/ --max_image_size 512 --min_image_size 256 --valid_ids 0 1 --num_proc 2

if not os.path.exists(os.path.join(DATA_DIR, "raw")):
    !python micro_diffusion/datasets/prepare/jdb/download.py --datadir $DATA_DIR/jdb/ --max_image_size 512 --min_image_size 256 --valid_ids 0 --num_proc 8

## Convert dataset to mds format

In [ ]:
# !python convert.py --images_dir ./datadir/jdb/raw/train/imgs/ --captions_jsonl ./datadir/jdb/raw/train/train_anno_realease_repath.jsonl --local_mds_dir ./datadir/jdb/mds/train/
if not os.path.exists(os.path.join(DATA_DIR, "jdb", "mds", "train")):
    !python micro_diffusion/datasets/prepare/jdb/convert.py --images_dir $DATA_DIR/jdb/raw/train/imgs/ --captions_jsonl $DATA_DIR/jdb/raw/train/train_anno_realease_repath.jsonl --local_mds_dir $DATA_DIR/jdb/mds/train/

In [ ]:
# !python convert.py --images_dir ./datadir/jdb/raw/valid/imgs/ --captions_jsonl ./datadir/jdb/raw/valid/valid_anno_repath.jsonl --local_mds_dir ./datadir/jdb/mds/valid/

if not os.path.exists(os.path.join(DATA_DIR, "jdb", "mds", "valid")):
    !python micro_diffusion/datasets/prepare/jdb/convert.py --images_dir $DATA_DIR/jdb/raw/valid/imgs/ --captions_jsonl $DATA_DIR/jdb/raw/valid/valid_anno_repath.jsonl --local_mds_dir $DATA_DIR/jdb/mds/valid/

## Precompute latents

In [ ]:
# for split in train valid; do
#     accelerate launch --multi_gpu --num_processes 8 precompute.py --datadir ./datadir/jdb/mds/$split/ --savedir ./datadir/jdb/mds_latents_sdxl1_dfnclipH14/$split/ --vae stabilityai/stable-diffusion-xl-base-1.0 --text_encoder openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378 --batch_size 32
# done
if not os.path.exists(os.path.join(DATA_DIR, "mds_latents_sdxl1_dfnclipH14", "train")):
    !WORLD_SIZE=1 RANK=0 python micro_diffusion/datasets/prepare/jdb/precompute.py --datadir $DATA_DIR/jdb/mds/train/ --savedir $DATA_DIR/mds_latents_sdxl1_dfnclipH14/train/ --vae stabilityai/stable-diffusion-xl-base-1.0 --text_encoder openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378 --batch_size 16

## Step1

In [ ]:
# !composer train.py --config-path ./configs --config-name res_256_pretrain.yaml exp_name=MicroDiTXL_mask_75_res_256_pretrain model.train_mask_ratio=0.75
# !composer train.py --config-path ./configs --config-name res_512_pretrain.yaml exp_name=MicroDiTXL_mask_75_res_512_pretrain model.train_mask_ratio=0.75
!composer train.py --config-path ./configs --config-name res_256_pretrain_test.yaml exp_name=MicroDiTXL_mask_75_res_512_pretrain model.train_mask_ratio=0.75

# parallelism_config: nullに変更
# nu_workers: 0に変更
# prefetch_factor: nullに変更
# persistent_workers: falseに変更